# 3 监督学习：数学原理
## 3.1 监督学习
监督学习是处理 $y = F(x)$ 中 $y$ 已知，求解功能函数 $F$ 的问题。根据 $y$ 的差异划分为：
- 回归：$y$ 是连续值或不可枚举的离散值。
- 分类：$y$ 是可枚举值的离散值。

多数监督学习算法都同时支持处理回归&分类问题。
## 3.2 回归
### 3.2.1 损失函数
**平均偏差误差**：$ MBE = \frac{1}{n}\sum_{i = 1}^n(y_i - \hat y_i) $ 
- 缺点：受到误差方向的迎向，正负抵，精确度不足。

**平均绝对值误差（L1 loss）**：$ MAE = \frac{1}{n}\sum_{i = 1}^n|y_i - \hat y_i| $
- 优点：鲁棒性好，少数异常点对终值影响较小。
- 缺点：零点不可导，可能存在多解；损失值偏差较小的与偏差较大的共享梯度，需要使用可变的学习率优化。

**均方根误差（L2 loss）**：$ RMSE = \sqrt {MSE} = \sqrt {\frac{1}{n}\sum_{i = 1}^n(y_i - \hat y_i)^2} $
- 优点：函数处处可导，有相对稳定的解。
- 缺点：异常值（误差超过1）敏感，会牺牲正常点的预测效果以优化权重更高的离群点，可用导数阈值优化。

**平滑绝对误差**：$ Huber loss = \sum_{i=1}^n \begin{cases} 0.5*(y_i - \hat y_i)^2 & |y_i - \hat y_i|<\delta \\ \delta * (|y_i - \hat y_i|-0.5 \delta ^2) & else \end{cases}$
- 小于 $\delta$ 为L2范数，其余为L1范数，让预测值与真实值差别过大时，梯度值不至于过大，反之又足够小。
- smooth L1($\delta = 1$) $ = \sum_{i=1}^n \begin{cases} 0.5*(y_i - \hat y_i)^2 & |y_i - \hat y_i|<1 \\ |y_i - \hat y_i|-0.5 & else \end{cases}$

**Log-Cosh Loss**：$ \sum_{i=1}^n log(cosh(y_i - \hat y_i)) $
- 对于较小的$x$，$log(cosh(x))$近似于$\frac{(x^2)}{2}$，对于较大的$x$，近似于$|x|-log(2)$。
- 优点：具有Huber loss所有的优点，预测误差的双曲余弦对数；优势是二阶导数处处可微。
- 缺点：误差极大时，一阶梯度和Hessian会变成定值，导致如XGBoost出现缺少分裂点的情况。

**分位数损失**：$ Quantile Loss = \sum_{y_i<\hat y_i}(1-\lambda)|y_i-\hat y_i| + \sum_{y_i\geq\hat y_i}\lambda|y_i - \hat y_i|, \lambda \in [0,1] $
- 面对不太线性的现实问题中，偏向针对某一区间预测的预测效果的解决方案。
- 基于对正误差和反误差的重视程度，通过$\lambda$（分位值）对偏高预测值与偏低预测值给予不同的惩罚，让函数对中值产生偏移。

以上只是回归问题常用的损失函数，当然还有许多损失函数未提及，也可以根据实际问题自定义损失函数。
### 3.2.2 寻解算法：最小二乘法（OLS）
**最小二乘法（最小平方法）**：通过最小化误差的平方和寻找数据的最佳匹配函数。对于求解最小二乘法结果的方法有**矩阵解法**和**梯度下降法**。

*以前我以为最小二乘法和梯度下降法是两个方法，其实梯度下降法只是最小二乘法求解的方法之一。*
#### 3.2.2.1 矩阵解法
为方便后续讲解，做出如下前提假设：
- **模型假设**：$ F(x)=a_1*x_1+a_2*x_2+……+a_n*x_n+b=[x_1,x_2,……,x_n,1][a_1,a_2,……,a_n,b]^T=XA $
- **损失函数**：$ L(x,a)=\frac{1}{2n}\sum_{i=1}^n[(a_1*x_{i1}+a_2*x_{i2}+……+a_n*x_{in}+b)-y_i]^2=\frac{1}{2n}(XA-Y)^T(XA-Y) $

依据假设，当损失函数的一阶偏导数（$\partial A$）为为零时，损失函数取最小值，即：
$$ \frac{\partial L}{\partial A}=\frac{\partial L}{\partial a_1}+\frac{\partial L}{\partial a_2}+……+\frac{\partial L}{\partial a_n}=\frac{1}{n}\sum_{i=1}^n(a_1x_{i1}+a_2x_{i2}+……+a_nx_{in})*(x_{i1}+x_{i2}+……+x_{in})=\frac{X^T(XA-Y)}{n}=0 $$
求解：
$$ A = (X^TX)^{-1}X^TY $$
以上的推导过程表明矩阵解法有如下限制：
- 1.系数矩阵$A$必须存在逆矩阵（可舍弃边缘特征使矩阵可逆缓解）。
- 2.系数矩阵$A$的求逆过程的耗时受矩阵规模影响（利用降维处理缓解）。
- 3.模型假设若非线性，求解将十分复杂或难以开展（可以转化为线性关系缓解）。

*（）中是一些预处理方案，优化矩阵解法的适配问题。*
#### 3.2.2.2 梯度下降法
梯度下降法是利用梯度逐步迭代模型参数，使损失函数达到最小值的方法。举例如下：
```
想象你和贝爷一起进行丛林探险，露宿山崖边缘，晨间丛林里雾气弥漫，你们选择下至山下的峡谷补充水源。此时，你们**各选了一个位置**开始下山，你用脚一点点感受被踩踏的岩石，选择合适的落脚点，你总是**优先选择垂直向下（即坡度最陡）**的方向下山，重复如此，最终你们下到了山脚。
```
梯度下降法的过程与之相似，使用相同的假设表达如下：
- **模型假设**：$ F(x)=XA $ 并随机初始化$A$中参数。
- **损失函数**：$ L(x,a)=\frac{1}{2n}(XA-Y)^T(XA-Y) $。
- **偏导数**：$ \frac{\partial L}{\partial A}=\frac{1}{n}X^T(XA-Y) $。
- **梯度下降**：$ A=A-\eta \frac{\partial L}{\partial A},\eta \in (0,1) $。
- **终止条件**：梯度值为零或不可改善时，则达到损失函数最小值，此时的模型参数为最优参数。

学习率（$\eta$）与梯度（$\frac{\partial L}{\partial A}$）共同决定每轮参数矩阵迭代的步长，其中学习率是用户设定值，需注意：
- 模型训练前应特征缩，提高$\eta$作用效果，高效下降。
- $\eta$设定过小，耗时严重；设定过大，反复跳跃难收敛。

与矩阵的解法中求解一阶导数为零不同，由于训练数据$X$，随机参数$A$、学习率$\eta$均为已知项，其目标是将现有的随机参数不断改善成最优函数的接近值，所以突破了矩阵解法的限制，更加通用。但是梯度下降法的主要挑战是复杂的损失函数，除了全局最小值外，还存在多个局部最小值，有时可能永远到不了全局最小值。
在实际使用梯度下降法时，由于计算梯度方式的差异，衍生出三种方法：
- **批梯度下降（BGD）**：每轮迭代计算梯度时使用全部训练数据；优点是朝着最小值迭代，缺点是样本值很大时每轮更新速度会很慢。
- **随机梯度下降（SGD）**：每轮迭代计算梯度时随机抽取一条训练数据；优点是每轮更新速度快，缺点是样本噪点较多时未必朝着极小值方向更新（多轮迭代能保障整体大致朝着极小值方向更新）。
- **小批量梯度下降（MBGD）**：每轮迭代计算梯度时随机抽取一定比例训练数据；优点是平衡更新梯度方向准确性与更新速度，缺点是batch值较难确定。

虽然随机性可以很好的跳过局部最优值，所以比较推荐使用小批量梯度下降法。由于随机梯度下降较难能达到最小值，可使用逐渐降低学习率缓和这一问题，这个过程被称为模拟退火。

以上虽然只使用了相对简单的假设进行举例，但相同的梯度下降法可以推广到更高维，更复杂的损失函数中使用。
## 3.3 分类
分类问题常见的是二分类问题（比如垃圾邮件识别），但也可以拥有任意数量的类，属于多类分类问题；前者模型称作二分类器，后者称作多类分类器（多项式分类器）。

处理多分类问题时，如随机森林、朴素贝叶斯等分类器支持多分类问题；而SVM 分类器、线性分类器等严格的二分类器，可使用如下策略：
- **一对所有（OvA）**：创建与类别数$N$相等的分类器数，每个二分类器训练时只设定其中一个类别为正例，其余均为负例，选出决策分数最高的分类器所对应的类。
- **一对一（OvO）**：将类别数量$N$两两配对作为训练子集单独训练二分类器，共计训练$\frac{N*(N-1)}{2}$个分类器，选出全部分类器中类别胜出最多的类。
### 3.3.1 损失函数
#### 3.1.1.1 均方误差
均方误差可以作为损失函数，但改问题下均方误差函数一般有多个局部最优解，计算时间长等问题，实用性不如后续的损失函数。
### 3.3.1.2 负似然对数
要理解负似然对数损失函数，需要先理解**概率与似然函数**的关系，假设：
- 某事件有且仅有正反例互斥的对立结果，比如是否是垃圾邮件问题中的是或否。
- 记事件正例发生的概率为 $p$,则对立事件概率为 $1-p$。
- 假设对实验总共进行了$N$次实验，其中$m$次出现正例，$n$出现返例，$N = m + n$。
- 实验结果的似然率是：$L(p)=p^m*(1-p)^n$，事件发生是有序的，故无排列组合系数,这就是似然函数。

简单而言，似然函数就是各实验结果所对应的事件概率的连乘积，且由于实验结果已发生，故使只有真实的概率才能使似然函数最大，使该实验结果产生。基于此，将$p$替换为$h_\theta(x)$：
$$ p(\theta) = \begin{cases} h_\theta(x) & y=1 \\ 1-h_\theta(x) & y=0 \end{cases} = h_\theta(x)^y + (1-h_\theta(x))^{(1-y)} $$
则似然函数为：
$$ L(\theta) = \prod_{i=1}^n h_\theta(x)^{y_i} + (1-h_\theta(x))^{(1-y_i)}  $$
由于使似然函数最大的$h_\theta(x)$中的$\theta$就是模型所需的参数，故使用负似然函数便是求解最小值，可做为损失函数：
$$ cost(\theta) = -L(\theta) = - \prod_{i=1}^n h_\theta(x)^{y_i} + (1-h_\theta(x))^{(1-y_i)} $$
上述损失函数的求导颇为复杂，对$ L(\theta) $取对数并不影响曲线单调性，即最小值的取值点不变，故可优化为：
$$ cost(\theta) = -lnL(\theta,x) = - \sum_{i=1}^n y_i * ln(h_\theta(x)) + (1-y_i)*ln(1-h_\theta(x)) $$
转化为矩阵公式：
$$ cost(\theta) = -Y^T * ln(h_\theta(X))-(E - Y)^T * (E - ln(h_\theta(X))) $$
### 3.3.3 交叉信息熵(cross entropy)
下面将会使用比较糙的个人理解来论述交叉信息熵，准确的概念可以查阅信息论相关文档。

**信息**：是一种价值度量，可以不恰当的用新闻来类比，小概率发生的大事件才是头条，信息价值高；所以在一个系统中，小概率事件的信息值高。
那么怎么度量这个信息值呢？以二叉树分支策略决策为例，对于推测某个事件的发生，总是先推测最可能发生的逐层递减，在这一个准则下，确定某个事件结果发生所消耗的决策路径长度就是信息，计算公式：$log_2(\frac{1}{p})$，$p$ 即为某事件的概率（此处我推测$log_n(\frac{1}{p})$,$n$为分叉数）。

**信息熵**：分类系统中存在多个类的结果，其信息的加权和：$ \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) $。

**交叉熵**：基于推测数据分布：$q_i$的加权：$ \sum_{i=1}^n p_i * log_2(\frac{1}{q_i}) $。

**相对熵**：$ 相对熵 = 交叉熵 - 信息熵 = \sum_{i=1}^n p_i * log_2(\frac{p_i}{q_i})$

所以损失函数可以使用相对熵，目标是使得相对熵最小，$ cost(\theta) = \sum_{i=1}^n p_i * log_2(\frac{1}{q_i}) - \sum_{i=1}^n p_i * log_2(\frac{1}{p_i}) $，由于信息熵是常量，故损失函数可以精简为：$ cost(\theta) = \sum_{i=1}^n p_i * log_2(\frac{1}{q_i})= -\sum_{i=1}^n p_i * log_2(q_i) $

令$q_i$为模型$h_\theta(x)$预测结果,则损失函数为：
$$ cost(\theta) = -\sum_{i=1}^n p_i * log_2(q_i) = -\sum_{i=1}^n p_i * [y_i * log_2(h_\theta(x)) + (1-y_i) * log_2(1 - h_\theta(x))] $$
### 3.3.4 指数损失(exponential loss)
*暂时还没研究透彻，adaboost的损失函数*
### 3.3.5 Hinger损失
*暂时还没研究*
### 3.3.7 感知损失（Hinger损失的特例）
*暂时还没研究*
### 3.3.8 铰链损失
*暂时还没研究*
### 3.3.2 寻解算法
由于函数的计算的复杂性，可导性问题，主要使用梯度下降，参见3.2.2.2（负似然对数与交叉信息熵的梯度形式：$X^T*(h_\theta(x)-Y)$）。

---
参考资料：
- [Sklearn 与 TensorFlow 机器学习实用指南](https://www.bookstack.cn/books/hands_on_Ml_with_Sklearn_and_TF)
- [最小二乘法](https://zhuanlan.zhihu.com/p/38128785)
- [极大似然估计与最大后验概率估计](https://zhuanlan.zhihu.com/p/40024110)
- [如何通俗的解释交叉熵与相对熵](https://www.zhihu.com/question/41252833)
- [波士顿房价预测任务](https://www.paddlepaddle.org.cn/tutorials/projectdetail/1515038#anchor-13)